In [147]:
import numpy as np
from decimal import Decimal
from math import log
import random
import copy

In [76]:
#initialize variables
N = 10
init_nrg = 1
release_coef = 1
N_moves = 10
early_stop = 10

In [162]:
#initialize universum (intermittent type)
U = [[i, i, init_nrg, 0] if i % 2 else [i, i, init_nrg, 1] for i in range(N)]
print(U)

[[0, 0, 1, 1], [1, 1, 1, 0], [2, 2, 1, 1], [3, 3, 1, 0], [4, 4, 1, 1], [5, 5, 1, 0], [6, 6, 1, 1], [7, 7, 1, 0], [8, 8, 1, 1], [9, 9, 1, 0]]


In [130]:
def get_form(U):
    '''retrive form'''
    form_state = []
    for bitom in U:
        form_state.append(bitom[3])
    return form_state

In [131]:
def bin_deriv(arr):
    arr1 = arr[1:]
    arr2 = arr[:-1]
    output_arr = []
    for i in range(len(arr)-1):
        x1 = arr1[i]
        x2 = arr2[i]
        output_arr.append(bool(x1) != bool(x2))
    return output_arr

def bin_deriv_k(arr, k):
    return arr if k == 0 else bin_deriv(bin_deriv_k(arr, k-1))

def bien(arr):
    t = Decimal(0)
    s_k = arr
    for k in range(len(arr)-1):
        ones = s_k.count(1)
        n = len(s_k)
        p = float(ones)/n
        e = 0 if p == 0 else -p*log(p, 2)
        g = 0 if p == 1 else -(1-p)*log(1-p, 2)
        t_k = Decimal(e+g)*Decimal(2**k)
        t += t_k
        s_k = bin_deriv(s_k)
    return float(t/Decimal(2**(len(arr)-1)-1))

def tbien(arr):
    l = 0
    t = 0
    if len(arr) == 1:
        raise ValueError(
            'the input string is too short')
    s_k = arr
    for k in range(len(arr)-1):
        ones = s_k.count(1)
        n = len(s_k)
        p = float(ones)/n
        e = 0 if p == 0 else -p*log(p, 2)
        g = 0 if p == 1 else -(1-p)*log(1-p, 2)
        l_k = log(k+2, 2)
        t_k = (e+g)*l_k
        l += l_k
        t += t_k
        s_k = bin_deriv(s_k)
    return (1./l)*t

In [132]:
def calculate_entropy(form_state):
    '''entropy calculation function'''
    return tbien(form_state) #placeholder

In [133]:
old_entropy = calculate_entropy(get_form(U))
print(old_entropy)
print(get_form(U))

0.045890376550990146
[1, 0, 1, 0, 1, 0, 1, 0, 1, 0]


In [145]:
def move(arr):
    '''make jumps and re-arrange''' #have a feeling it works in-place
    arr_new = copy.deepcopy(arr)
    for i, bitom in enumerate(arr_new):
        old_pos = bitom[1] = i
        energy = bitom[2]
        magnitude = random.randrange(energy+1)
        direction = random.randrange(2)*2 - 1
        jump = direction * magnitude
        new_pos = (old_pos + jump) % N
        remain_nrg = energy - magnitude
        bitom[2] = remain_nrg
        bitom[0] = new_pos
    return sorted(arr_new)

In [155]:
#move one step
U_test = move(U)
print(U_test)
print(get_form(U_test))

[[1, 2, 0, 1], [2, 1, 0, 0], [3, 3, 1, 0], [3, 4, 0, 1], [4, 5, 0, 0], [6, 6, 1, 1], [7, 7, 1, 0], [7, 8, 0, 1], [9, 0, 0, 1], [9, 9, 1, 0]]
[1, 0, 0, 1, 0, 1, 0, 1, 1, 0]


In [153]:
#calculate new entropy and energy release
new_entropy = calculate_entropy(get_form(U))
entropy_change = new_entropy - old_entropy
nrg_release = entropy_change * release_coef

In [88]:
def energy_allocation(form_state):
    '''calculate distribution of energy change
    depending on substring entropies'''
    arr = form_state
    master_array = np.array([0]*N)
    for conv_len in range(2, N):
        for i in range(N):
            aux_array = [0]*N
            start = i
            end = i + conv_len
            if end > N:
                sub_arr = arr[i:] + arr[:end-N]
                entropy = calculate_entropy(sub_arr)
                aux_array[i:] = [entropy]*(N-i)
                aux_array[:end-N] = [entropy]*(end-N)
            else:
                sub_arr = arr[start:end]
                entropy = calculate_entropy(sub_arr)
                aux_array[start:end] = [entropy]*conv_len
            master_array = master_array + np.array(aux_array)
    if master_array.sum() != 0: #when can it be equal to zero?
        master_array = master_array / master_array.sum()
    return master_array

In [89]:
#energy release per bitom
nrg_release_bitom = nrg_release * energy_allocation(get_form(U))

In [90]:
def get_energy(U):
    '''retrieve energy'''
    energy_state = []
    for bitom in U:
        energy_state.append(bitom[2])
    return energy_state

In [92]:
#calculate new energy state
energy_state = np.array(get_energy(U))
energy_state = energy_state + np.array(nrg_release_bitom)

In [93]:
def set_energy(energy_state, U):
    for i, bitom in enumerate(U):
        bitom[2] += energy_state[i]

In [94]:
#pack new energy back
set_energy(energy_state, U)

In [166]:
def create_U(N, init_nrg, init_mode):
    '''create universum with minumum entropy state
    of a certain kind'''
    assert init_mode in ['interm', 'even']
    i_e = init_nrg
    if init_mode == 'interm':
        U = [[i, i, i_e, 0] if i % 2 else [i, i, i_e, 1] for i in range(N)]
    else:
        U = [[i, i, i_e, 0] if i < N/2 else [i, i, i_e, 1] for i in range(N)]
    return U

In [171]:
#workable loop
N = 20
init_nrg = 1
release_coef = 1
N_moves = 10
early_stop = 10
init_mode = 'even'

U = create_U(N, init_nrg, init_mode)

for t in range(10):
    old_entropy = calculate_entropy(get_form(U))
    es_counter = 0 #early stop counter
    entropy_change = 0
    while entropy_change <= 0 and es_counter < early_stop:
        U_new = move(U)
        new_entropy = calculate_entropy(get_form(U_new))
        entropy_change = new_entropy - old_entropy
        es_counter += 1
    if es_counter == early_stop:
        print('early stop')
        break
    U = U_new
    print(t, get_form(U))
    nrg_release = entropy_change * release_coef
    nrg_release_bitom = nrg_release * energy_allocation(get_form(U))
    energy_state = np.array(get_energy(U))
    energy_state = energy_state + np.array(nrg_release_bitom)
    energy_state = energy_state.astype(int)
    set_energy(energy_state, U)

0 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
1 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1]
2 [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1]
3 [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1]
early stop
